#### 11.벡터 검색.md

In [1]:
# !pip install -qU pinecone openai
import os
from dotenv import load_dotenv
load_dotenv()  # .env 파일의 환경변수 로드

# Pinecone 연결 및 인덱스 초기화
import pinecone
from pinecone import Pinecone as PineconeClient

# API 키와 환경명 가져오기
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from pinecone import Pinecone, ServerlessSpec

# API 키 설정 (본인의 API 키로 교체하세요)
pcone = Pinecone(api_key=PINECONE_API_KEY)   # Pinecone API 키 입력

# Pinecone 인덱스 생성
index_name = "movie-vector-index"  # 인덱스 이름 설정
if not pcone.has_index(index_name):
    pcone.create_index(
        name=index_name,
        dimension=1536,            # 임베딩 벡터 차원 (모델 output 크기와 맞춤)
        metric="cosine",           # 유사도 metric 선택
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
# 생성된 인덱스에 연결
index = pcone.Index(index_name)

In [3]:
# 예시 한글 영화 데이터셋 정의
movies = [
    {
        "id": "movie1",
        "title": "7번방의 선물",
        "year": 2013,
        "genre": "드라마",
        "description": "억울한 누명을 쓰고 교도소에 수감된 아빠와 그의 어린 딸의 감동적인 스토리"
    },
    {
        "id": "movie2",
        "title": "미나리",
        "year": 2020,
        "genre": "드라마",
        "description": "한국계 미국인 가족의 따뜻하고 감성적인 성장 이야기"
    },
    {
        "id": "movie3",
        "title": "기생충",
        "year": 2019,
        "genre": "드라마",
        "description": "가난한 가족과 부자 가족 사이의 빈부격차를 그린 사회 풍자 드라마"
    },
    {
        "id": "movie4",
        "title": "범죄도시",
        "year": 2017,
        "genre": "범죄",
        "description": "형사가 범죄 조직을 소탕하는 범죄 액션 영화"
    },
    {
        "id": "movie5",
        "title": "범죄도시 2",
        "year": 2022,
        "genre": "범죄",
        "description": "형사와 범죄 조직의 대결을 그린 범죄 액션 영화의 속편"
    },
    {
        "id": "movie6",
        "title": "헤어질 결심",
        "year": 2022,
        "genre": "범죄",
        "description": "산에서 발생한 의문의 죽음(살인 사건)을 수사하던 형사가 피의자에게 이끌리며 벌어지는 미스터리 멜로 영화"
    },
    {
        "id": "movie7",
        "title": "다만 악에서 구하소서",
        "year": 2020,
        "genre": "범죄",
        "description": "청부 살인업자와 범죄 조직의 마지막 거래를 그린 범죄 액션 영화"
    }
]

In [4]:
from langchain_openai import OpenAIEmbeddings

openai_embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",  # OpenAI 임베딩 모델 선택
    api_key=OPENAI_API_KEY  # OpenAI API 키 설정
)

In [5]:
# 모든 영화 설명에 대해 임베딩 벡터 생성
descriptions = [movie["description"] for movie in movies]        # 설명문 리스트
movie_vectors = openai_embeddings.embed_documents(descriptions)         # 각 설명문에 대한 임베딩 벡터 리스트 생성

# 임베딩 벡터의 차원 확인 (예상: 1536차원)
print(f"임베딩 벡터 차원: {len(movie_vectors[0])}")

임베딩 벡터 차원: 1536


In [6]:
# Pinecone에 벡터 업로드 (upsert)
# 각 레코드는 (id, 벡터, metadata)의 형태로 준비
vector_data = []
for movie, vector in zip(movies, movie_vectors):
    # metadata로 title, genre, year, description 저장
    meta = {
        "title": movie["title"],
        "genre": movie["genre"],
        "year": movie["year"],
        "description": movie["description"]
    }
    vector_data.append((movie["id"], vector, meta))

# 벡터들을 Pinecone 인덱스에 업서트
index.upsert(vectors=vector_data)

{'upserted_count': 7}

In [ ]:
# 검색 쿼리 예시
query = "감성적인 드라마 영화 추천해줘"

# 쿼리 임베딩 생성
query_vector = openai_embeddings.embed_query(query)
# Pinecone에서 벡터 유사도 검색 수행 (코사인 유사도 기반)
# 상위 3개의 가장 가까운 벡터를 찾고, 메타데이터를 포함하여 반환
result = index.query(vector=query_vector, top_k=3, threshold=0.5
                     , filter=None, include_metadata=True)

# 결과 출력: 각 결과의 제목, 연도, 장르를 표시
for match in result["matches"]:
    info = match["metadata"]
    print(f"{info['title']} - {info['year']} ({info['genre']}) : , Score: {match['score']:.4f}")

기생충 - 2019.0 (드라마) : , Score: 0.3915
7번방의 선물 - 2013.0 (드라마) : , Score: 0.3618
범죄도시 - 2017.0 (범죄) : , Score: 0.3376


In [12]:
# result?
len(result["matches"])

3

In [8]:
# 하이브리드 검색 예시: "2020년 이후의 범죄 영화 보여줘"
hybrid_query = "2020년 이후의 범죄 영화 보여줘"
hybrid_query = "2022년 이후의 범죄 영화 보여줘"

# 쿼리 임베딩 벡터 생성
hybrid_vector = openai_embeddings.embed_query(hybrid_query)

# 필터 조건 설정: 2020년 이후 & 장르 범죄
hybrid_filter = {
    # "year": {"$gte": 2020},
    "year": {"$gte": 2022},
    "genre": {"$eq": "범죄"}
}

# Pinecone 검색 실행 (쿼리+필터)
# threshold 값을 적절히 조정하려면, 결과의 score 분포를 확인하고 원하는 상위 결과만 필터링할 수 있습니다.
# 예를 들어, 상위 결과 중 score가 0.3 이상인 것만 보고 싶다면 threshold=0.3으로 설정합니다.
# 여러 쿼리에서 score 분포를 확인한 뒤, 실제로 원하는 결과가 포함되는 최소 score를 threshold로 정하면 됩니다.

# 예시: score가 0.3 이상인 결과만 반환
result3 = index.query(
    vector=hybrid_vector,
    top_k=3,
    threshold=0.3,  # 원하는 최소 score로 조정
    filter=hybrid_filter,
    include_metadata=True
)

for match in result3["matches"]:
    info = match["metadata"]
    print(f"{info['title']} - {info['year']} ({info['genre']}) : , Score: {match['score']:.4f}")

범죄도시 2 - 2022.0 (범죄) : , Score: 0.5317
헤어질 결심 - 2022.0 (범죄) : , Score: 0.2816
